In [1]:
import numpy as np
import pandas as pd

In [2]:
timesteps = 14
startDay = 350

In [3]:
dt = pd.read_csv("m5-forecasting-accuracy/sales_train_evaluation.csv")
dt.head(3)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1


In [4]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int64(1941), object(6)
memory usage: 452.9+ MB


In [5]:
#To reduce memory usage
def downcast_dtypes(df):
    float_cols = [c for c in df if df[c].dtype == "float64"]
    int_cols = [c for c in df if df[c].dtype in ["int64", "int32"]]
    df[float_cols] = df[float_cols].astype(np.float32)
    df[int_cols] = df[int_cols].astype(np.int16)
    return df

In [6]:
#Reduce memory usage and compare with the previous one to be sure
dt = downcast_dtypes(dt)

In [7]:
dt.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 30490 entries, 0 to 30489
Columns: 1947 entries, id to d_1941
dtypes: int16(1941), object(6)
memory usage: 114.3+ MB


In [8]:
#Take the transpose so that we have one day for each row, and 30490 items' sales as columns
dt = dt.T    
dt.head(8)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
id,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_006_CA_1_evaluation,HOBBIES_1_007_CA_1_evaluation,HOBBIES_1_008_CA_1_evaluation,HOBBIES_1_009_CA_1_evaluation,HOBBIES_1_010_CA_1_evaluation,...,FOODS_3_818_WI_3_evaluation,FOODS_3_819_WI_3_evaluation,FOODS_3_820_WI_3_evaluation,FOODS_3_821_WI_3_evaluation,FOODS_3_822_WI_3_evaluation,FOODS_3_823_WI_3_evaluation,FOODS_3_824_WI_3_evaluation,FOODS_3_825_WI_3_evaluation,FOODS_3_826_WI_3_evaluation,FOODS_3_827_WI_3_evaluation
item_id,HOBBIES_1_001,HOBBIES_1_002,HOBBIES_1_003,HOBBIES_1_004,HOBBIES_1_005,HOBBIES_1_006,HOBBIES_1_007,HOBBIES_1_008,HOBBIES_1_009,HOBBIES_1_010,...,FOODS_3_818,FOODS_3_819,FOODS_3_820,FOODS_3_821,FOODS_3_822,FOODS_3_823,FOODS_3_824,FOODS_3_825,FOODS_3_826,FOODS_3_827
dept_id,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,HOBBIES_1,...,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3,FOODS_3
cat_id,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,HOBBIES,...,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS,FOODS
store_id,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,CA_1,...,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3,WI_3
state_id,CA,CA,CA,CA,CA,CA,CA,CA,CA,CA,...,WI,WI,WI,WI,WI,WI,WI,WI,WI,WI
d_1,0,0,0,0,0,0,0,12,2,0,...,0,14,1,0,4,0,0,0,0,0
d_2,0,0,0,0,0,0,0,15,0,0,...,0,11,1,0,4,0,0,6,0,0


In [9]:
#Remove id, item_id, dept_id, cat_id, store_id, state_id columns
dt = dt[6 + startDay:]
dt.head(5)

,0,1,2,3,4,5,6,7,8,9,...,30480,30481,30482,30483,30484,30485,30486,30487,30488,30489
d_351,0,0,0,2,0,0,0,24,3,2,...,0,9,1,0,11,0,0,1,0,0
d_352,0,0,0,0,0,0,0,9,0,2,...,0,5,4,0,8,0,1,2,0,0
d_353,0,0,0,4,2,0,0,2,1,1,...,0,15,2,0,3,0,1,2,0,0
d_354,0,1,0,2,0,0,0,7,1,0,...,0,5,1,0,3,0,0,0,0,0
d_355,0,0,0,1,2,0,0,0,0,0,...,0,7,1,0,1,0,1,1,0,0


In [10]:
calendar = pd.read_csv("m5-forecasting-accuracy/calendar.csv")
calendar

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1964,2016-06-15,11620,Wednesday,5,6,2016,d_1965,NaN,NaN,NaN,NaN,0,1,1
1965,2016-06-16,11620,Thursday,6,6,2016,d_1966,NaN,NaN,NaN,NaN,0,0,0
1966,2016-06-17,11620,Friday,7,6,2016,d_1967,NaN,NaN,NaN,NaN,0,0,0
1967,2016-06-18,11621,Saturday,1,6,2016,d_1968,NaN,NaN,NaN,NaN,0,0,0


In [11]:
#Create dataframe with zeros for 1969 days in the calendar
daysBeforeEvent = pd.DataFrame(np.zeros((1969,1)))

In [12]:
# "1" is assigned to the days before the event_name_1. Since "event_name_2" is rare, it was not added.
for x,y in calendar.iterrows():
    if((pd.isnull(calendar["event_name_1"][x])) == False):
           daysBeforeEvent[0][x-1] = 1 
            #if first day was an event this row will cause an exception because "x-1".
            #Since it is not i did not consider for now.

In [13]:
#"calendar" won't be used anymore. 
del calendar

In [14]:
#"daysBeforeEventTest" will be used as input for predicting (We will forecast the days 1913-1941)
daysBeforeEventTest = daysBeforeEvent[1941:1969]
#"daysBeforeEvent" will be used for training as a feature.
daysBeforeEvent = daysBeforeEvent[startDay:1941]

In [15]:
#Before concatanation with our main data "dt", indexes are made same and column name is changed to "oneDayBeforeEvent"
daysBeforeEvent.columns = ["oneDayBeforeEvent"]
daysBeforeEvent.index = dt.index

In [16]:
dt = pd.concat([dt, daysBeforeEvent], axis = 1)

dt.columns

Index([                  0,                   1,                   2,
                         3,                   4,                   5,
                         6,                   7,                   8,
                         9,
       ...
                     30481,               30482,               30483,
                     30484,               30485,               30486,
                     30487,               30488,               30489,
       'oneDayBeforeEvent'],
      dtype='object', length=30491)

In [17]:
dt

,0,1,2,3,4,5,6,7,8,9,...,30481,30482,30483,30484,30485,30486,30487,30488,30489,oneDayBeforeEvent
d_351,0,0,0,2,0,0,0,24,3,2,...,9,1,0,11,0,0,1,0,0,0.0
d_352,0,0,0,0,0,0,0,9,0,2,...,5,4,0,8,0,1,2,0,0,1.0
d_353,0,0,0,4,2,0,0,2,1,1,...,15,2,0,3,0,1,2,0,0,0.0
d_354,0,1,0,2,0,0,0,7,1,0,...,5,1,0,3,0,0,0,0,0,0.0
d_355,0,0,0,1,2,0,0,0,0,0,...,7,1,0,1,0,1,1,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
d_1937,0,0,0,1,0,0,1,5,0,1,...,6,3,0,0,1,0,1,0,0,0.0
d_1938,3,0,2,3,0,0,0,4,0,1,...,4,3,1,2,0,1,0,1,2,0.0
d_1939,3,0,3,0,2,5,1,1,0,0,...,4,3,1,1,0,0,1,1,2,0.0
d_1940,0,0,0,2,1,2,1,40,1,0,...,1,0,0,3,1,1,0,1,5,0.0


In [18]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Assuming dt is a pandas DataFrame
dt.columns = dt.columns.astype(str)  # Convert column names to strings

sc = MinMaxScaler(feature_range=(0, 1))
dt_scaled = sc.fit_transform(dt)

In [19]:
X_train = []
y_train = []
for i in range(timesteps, 1941 - startDay):
    X_train.append(dt_scaled[i-timesteps:i])
    y_train.append(dt_scaled[i][0:30490]) 

In [20]:
del dt_scaled

In [21]:
#Convert to np array to be able to feed the LSTM model
X_train = np.array(X_train)
y_train = np.array(y_train)
print(X_train.shape)
print(y_train.shape)

(1577, 14, 30491)
(1577, 30490)


In [22]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout

# Initialising the RNN
lstm = Sequential()

# Adding the first LSTM layer and some Dropout regularisation
layer_1_units=50
lstm.add(LSTM(units = layer_1_units, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
lstm.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
lstm.add(LSTM(units = layer_3_units, return_sequences = True))
lstm.add(Dropout(0.2))

# Adding a third LSTM layer and some Dropout regularisation
layer_3_units=400
lstm.add(LSTM(units = layer_3_units))
lstm.add(Dropout(0.2))

# Adding the output layer
lstm.add(Dense(units = 30490))

# Compiling the RNN
lstm.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
epoch_no=32
batch_size_RNN=44
lstm.fit(X_train, y_train, epochs = epoch_no, batch_size = batch_size_RNN)

Epoch 1/32
36/36 [==============================] - 7s 140ms/step - loss: 0.0180
Epoch 2/32
36/36 [==============================] - 5s 144ms/step - loss: 0.0156
Epoch 3/32
36/36 [==============================] - 5s 143ms/step - loss: 0.0151
Epoch 4/32
36/36 [==============================] - 5s 144ms/step - loss: 0.0149
Epoch 5/32
36/36 [==============================] - 5s 145ms/step - loss: 0.0147
Epoch 6/32
36/36 [==============================] - 5s 152ms/step - loss: 0.0145
Epoch 7/32
36/36 [==============================] - 6s 159ms/step - loss: 0.0143
Epoch 8/32
36/36 [==============================] - 6s 155ms/step - loss: 0.0142
Epoch 9/32
36/36 [==============================] - 7s 183ms/step - loss: 0.0141
Epoch 10/32
36/36 [==============================] - 6s 154ms/step - loss: 0.0140
Epoch 11/32
36/36 [==============================] - 5s 148ms/step - loss: 0.0139
Epoch 12/32
36/36 [==============================] - 6s 163ms/step - loss: 0.0138
Epoch 13/32
36/36 [======

In [23]:
inputs= dt[-timesteps:]
inputs = sc.transform(inputs)

In [24]:
X_test = []
X_test.append(inputs[0:timesteps])
X_test = np.array(X_test)
predictions = []

for j in range(timesteps,timesteps + 28):
#X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
    predicted_stock_price = lstm.predict(X_test[0,j - timesteps:j].reshape(1, timesteps, 30491))
    testInput = np.column_stack((np.array(predicted_stock_price), daysBeforeEventTest[0][1941 + j - timesteps]))
    X_test = np.append(X_test, testInput).reshape(1,j + 1,30491)
    predicted_stock_price = sc.inverse_transform(testInput)[:,0:30490]
    predictions.append(predicted_stock_price)

1/1 [==============================] - 0s 17ms/step


In [25]:
import time

submission = pd.DataFrame(data=np.array(predictions).reshape(28,30490))

submission = submission.T
    
submission = pd.concat((submission, submission), ignore_index=True)

sample_submission = pd.read_csv("m5-forecasting-accuracy/sample_submission.csv")
    
idColumn = sample_submission[["id"]]
    
submission[["id"]] = idColumn  

cols = list(submission.columns)
cols = cols[-1:] + cols[:-1]
submission = submission[cols]

colsdeneme = ["id"] + [f"F{i}" for i in range (1,29)]

submission.columns = colsdeneme

currentDateTime = time.strftime("%d%m%Y_%H%M%S")

submission

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.826349,0.831218,0.857881,0.902824,1.049808,1.414223,1.365165,0.872894,0.784138,...,0.822922,1.124783,1.295131,0.875967,0.686653,0.679992,0.642170,0.661164,0.890629,1.107610
1,HOBBIES_1_002_CA_1_validation,0.304081,0.263063,0.283457,0.291540,0.284755,0.273805,0.286752,0.293235,0.257785,...,0.323955,0.324126,0.301183,0.263359,0.233111,0.287986,0.322197,0.337856,0.351584,0.321041
2,HOBBIES_1_003_CA_1_validation,0.425636,0.386725,0.448790,0.540466,0.687802,0.916247,0.903489,0.459570,0.343940,...,0.534512,0.725347,0.788097,0.420042,0.247162,0.287669,0.344282,0.432407,0.569499,0.607147
3,HOBBIES_1_004_CA_1_validation,1.926798,1.310683,1.310971,1.520464,1.802822,2.405155,3.847824,2.348647,1.332428,...,1.757507,2.318666,3.808349,3.019732,1.366161,1.213298,1.393010,1.729894,2.316589,3.842311
4,HOBBIES_1_005_CA_1_validation,1.489929,1.368551,1.379341,1.394522,1.393987,1.423859,1.648778,1.462778,1.281009,...,1.194221,1.290559,1.520660,1.396319,1.022298,0.990823,0.995246,1.023433,1.191010,1.568366
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.554631,0.472840,0.465809,0.453575,0.467813,0.509804,0.735672,0.520817,0.395845,...,0.371674,0.414182,0.570152,0.478019,0.315704,0.325470,0.309429,0.294685,0.311607,0.391307
60976,FOODS_3_824_WI_3_evaluation,0.122850,0.168017,0.207433,0.223578,0.221834,0.156017,0.184047,0.158793,0.173738,...,0.235606,0.190893,0.140550,0.163769,0.169383,0.213457,0.228234,0.230686,0.188672,0.122016
60977,FOODS_3_825_WI_3_evaluation,0.872524,0.879254,0.816957,0.770571,0.736501,0.564100,0.646281,0.795414,0.802385,...,0.649802,0.572697,0.509324,0.643806,0.645255,0.620765,0.564337,0.584208,0.609845,0.584174
60978,FOODS_3_826_WI_3_evaluation,1.532276,1.484510,1.290017,1.114112,1.026204,1.038501,1.355364,1.444979,1.376078,...,0.967348,1.001645,1.249811,1.363038,1.184993,1.056029,0.911604,0.856681,0.871002,1.000551


In [26]:
submission.to_csv("submission_lstm.csv", index=False)